(1) I hereby agree to follow any and all assignment rules and procedures as stated in Canvas for this course, MATH2319.

(2) I solemnly swear that I will not discuss/ have not discussed my assignment solutions with anyone in any way and the solutions I am submitting are my own personal work.

Full Name: Luke Daws

# MATH2319 - Machine Learning

# Assignment 1

## Luke Daws

## S3322003

***

# Question 1

One of the most import aspect of any data analysis or machine learning task is data pre-processing. An important saying in the field is "Garbage in, garbage out". The first question of the assignment is to go through a dataset and clean it up ready to be processed.

### The data set

The dataset for that is used for this assignment is the `Credit Approval` data taken from the UCI repository:

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

From this repository we only require two of the supplied datasets:

- `crx.names`: This file contains information about the dataset itself, such as the names of the features, what values are in the features and their type (i.e. continuous, categorical), The number of instances for the features (690) and how many missing values there are in each feature.
- `crx.data`: This is the raw data. There are no headings supplied with this data set and it is separated by the `,` symbol. 



### Importing packages

We will be using 3 packages for this assignment.

- `Numpy`: This package is good for implementing array data structures performing calculations with the arrays.
- `Pandas`: A very important package that allows for easy data frame construction and manipulation. It also contains other tools for data analysis.
- `sklearn`: sklearn is used heavily in machine learning but for this assignment it is only going to be used for it's `preprocessing` package in question 1.


In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn import preprocessing

### Importing the dataset

I will be importing the dataset from Azure. Now by looking at the data in the dataset and reading the `crx.names` file we know that the data is separated by the `,` symbol and also that the missing values in the dataset are replaced by the `?` symbol.

In most cases the missing values will just not be there or they will be replaced with something else like `?` or `9999`. We will go through and check that these are the only cases of missing values later but as we know what they are now, we will add the argument `na_values=["?"]` to automatically turn them into `NaN`.

In some cases, you can be lucky and the feature names are already included in the dataset so when you import the data all the features will have the appropriate title. Unfortunately, this dataset doesn't so we need to make sure that the first row of data isn't considered a title. So, we will also use the argument `header=None`.


In [2]:
credit_data = pd.read_csv('Assignment1_Q1_crx.data', sep=',', header=None, na_values=["?"])

### Checking the data frame

Now that the data has been imported we will check to make sure it has been imported correctly.

In [3]:
credit_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


### Changing column names

As you can see the data has been imported correctly and the first row of data from `crx.data` has not been used for the titles of each feature, which is what we wanted, but they still need to be lablled. The `crx.name` file tells us that each feature is called `A1` all to `A16`. So, we will fix those titles now. 

In [4]:
credit_data.columns = ['A1', 'A2','A3', 'A4','A5', 'A6','A7', 'A8','A9', 'A10','A11', 'A12','A13', 'A14','A15', 'A16',]

In [5]:
credit_data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


### Checking for features with only one value

Features that only contain one value provided no insight into your data, so we need to remove any feature that contains only one value. `crx.name` tells us each value that is in each feature so we don't really need to run this code but on the off chance there is a feature with no unique values we will run it anyway.  

In [6]:
credit_data = credit_data.loc[:, credit_data.nunique() != 1]

In [7]:
credit_data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


As you can see there has been no change to the dataframe

## Data Cleaning

### Checking data type

We need to make sure that each feature is the correct data type as this can tell us that there is an issue we need to fix during this pre-processing stage. for example, a numerical only feature could be considered a float64 or int64 datatype but if it is shown as an object datatype that would mean a string is amongst that data.


In [8]:
credit_data.dtypes

A1      object
A2     float64
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11      int64
A12     object
A13     object
A14    float64
A15      int64
A16     object
dtype: object

Comparing what we are shown above with what the `crx.name` file tells us then all these data types are correct.

### Missing values (NAN, NULL) 

The next step is too look for missing values (`NaN` or `Null`). As we know when importing the dataset we converted the `?` into `null` values, so we know there should be some.

In [9]:
credit_data.isna().sum()

A1     12
A2     12
A3      0
A4      6
A5      6
A6      9
A7      9
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64

As we can see there are a few missing values and cross referencing with the `crx.names` file, we can see that the count of how many missing values there are and where they are located correspond with each other. We will be replacing these values later on.

Below we will do further investigation into the data set to find anything unusual about it that may need to be fixed before the analysts can begin.

In [10]:
credit_data.describe(include = np.number).round(2)

,A2,A3,A8,A11,A14,A15
count,678.00,690.00,690.00,690.00,677.00,690.00
mean,31.57,4.76,2.22,2.40,184.01,1017.39
std,11.96,4.98,3.35,4.86,173.81,5210.10
min,13.75,0.00,0.00,0.00,0.00,0.00
25%,22.60,1.00,0.16,0.00,75.00,0.00
50%,28.46,2.75,1.00,0.00,160.00,5.00
75%,38.23,7.21,2.62,3.00,276.00,395.50
max,80.25,28.00,28.50,67.00,2000.00,100000.00


In [11]:
credit_data.describe(include = np.object)

,A1,A4,A5,A6,A7,A9,A10,A12,A13,A16
count,678,684,684,681,681,690,690,690,690,690
unique,2,3,3,14,9,2,2,2,3,2
top,b,u,g,c,v,t,f,f,g,-
freq,468,519,519,137,399,361,395,374,625,383


Running this code provides a good overview of each feature. We can see in some cases that the count is lower which corresponds with the missing values that we checked for earlier. The main advantage of this description is looking for impossible or unusual values, especially the numerical values. For instance, if you have an age feature and notice that the max age is well above the normal age of a living person you can consider that value to be incorrect. Unfortunately, due to the data being anonymised we can’t 100% know what would be an unusual value or outlier for the supplied data.

If we did know the context for each feature, we could use boxplots to look for outliers or unusual values, as well as histograms and even selecting only unique values of each feature. I will provide an example of what you can do below but I will only do it for one feature as it is not necessary to run this on each feature for this dataset. 


In [12]:
credit_data.hist(column=['A14'])
A14 = credit_data['A14'].unique()
sorted(A14)

[0.0,
 20.0,
 21.0,
 24.0,
 29.0,
 30.0,
 40.0,
 43.0,
 49.0,
 50.0,
 52.0,
 60.0,
 70.0,
 75.0,
 80.0,
 93.0,
 94.0,
 96.0,
 100.0,
 102.0,
 112.0,
 120.0,
 128.0,
 129.0,
 140.0,
 144.0,
 145.0,
 150.0,
 152.0,
 160.0,
 164.0,
 167.0,
 168.0,
 171.0,
 176.0,
 180.0,
 181.0,
 188.0,
 195.0,
 200.0,
 202.0,
 204.0,
 212.0,
 216.0,
 239.0,
 240.0,
 250.0,
 260.0,
 268.0,
 274.0,
 280.0,
 288.0,
 300.0,
 311.0,
 320.0,
 329.0,
 350.0,
 360.0,
 368.0,
 370.0,
 375.0,
 380.0,
 381.0,
 393.0,
 395.0,
 396.0,
 399.0,
 400.0,
 408.0,
 410.0,
 420.0,
 431.0,
 434.0,
 440.0,
 443.0,
 455.0,
 491.0,
 500.0,
 510.0,
 515.0,
 520.0,
 583.0,
 711.0,
 nan,
 17.0,
 22.0,
 28.0,
 32.0,
 45.0,
 56.0,
 62.0,
 73.0,
 76.0,
 86.0,
 88.0,
 92.0,
 99.0,
 108.0,
 110.0,
 117.0,
 121.0,
 130.0,
 132.0,
 136.0,
 141.0,
 154.0,
 156.0,
 163.0,
 170.0,
 174.0,
 178.0,
 186.0,
 208.0,
 210.0,
 211.0,
 220.0,
 221.0,
 224.0,
 225.0,
 228.0,
 230.0,
 231.0,
 232.0,
 252.0,
 253.0,
 254.0,
 256.0,
 263.0,
 272.0,
 2

Again without knowing what the context is for each figure we have to assume that the supplied values are within the normal range.

Once we had found unusual values or outliers, there are a few ways we could deal with them. We could simply turn them in to `Null` values so we can replace them with something later or we could even consider removing that row completely.

The `crx.name` file has also supplied us with what the value should be within each categorical feature. So we will check to make sure there is a corresponding amount of unique values in the data set we imported.

In [13]:
credit_data.nunique()

A1       2
A2     349
A3     215
A4       3
A5       3
A6      14
A7       9
A8     132
A9       2
A10      2
A11     23
A12      2
A13      3
A14    170
A15    240
A16      2
dtype: int64

We can see the for each categorical feature (`A1`,`A4`,`A5`,`A6`,`A7`,`A9`,`A10`,`A12`,`A13`,`A16`) there is the correct amount of unique values, except for `A4`. In this case there is less unique values which without knowing the context of the feature is fine, even with context it could just mean no one falls under that category. So for this case nothing major needs to be done. 

## Replaceing missing values

As part of this assignent we are to replace the missing values with either the `mode`, for categorical features, or the `median` for the numerical features.

Starting with the categorical features:

In [14]:
credit_data[['A1','A4','A5','A6','A7']].mode()

,A1,A4,A5,A6,A7
0,b,u,g,c,v


This tell us what value is the mode for each feature we are looking at. 

In [15]:
credit_data['A1'].value_counts()

b    468
a    210
Name: A1, dtype: int64

Here we can see the unique value count in the feature. we can cross reference this with the supplied `crx.name` file. This is also useful for checking if there have been different variations of spellings for certain values, misspellings or even abbreviations (e.g. for gender: Male, male, M, etc…)

In [16]:
credit_data['A1'].replace([None], credit_data['A1'].mode(), inplace=True)
credit_data['A1'].value_counts()

b    480
a    210
Name: A1, dtype: int64

Now we can see that there are more `b` values than before showing that the `Null` values have been replaced. also adding them up (480 + 210 = 690) shows that it is the correct amount of values in the feature.

In [17]:
credit_data['A4'].value_counts()

u    519
y    163
l      2
Name: A4, dtype: int64

In [18]:
credit_data['A4'].replace([None], 'u', inplace=True)
credit_data['A4'].value_counts()

u    525
y    163
l      2
Name: A4, dtype: int64

In [19]:
credit_data['A5'].value_counts()

g     519
p     163
gg      2
Name: A5, dtype: int64

In [20]:
credit_data['A5'].replace([None], 'g', inplace=True)
credit_data['A5'].value_counts()

g     525
p     163
gg      2
Name: A5, dtype: int64

We could consider `gg` as a mistype and change that to the `g` value but without context of the feature we can't be sure that's not correct. also the `crx.name` file says that should be a variable.  

If the value `gg` was meant to be `g` we would still use the same code but use `credit_data['A5'].replace('gg', 'g', inplace=True)`

In [21]:
credit_data['A6'].value_counts()

c     137
q      78
w      64
i      59
aa     54
ff     53
k      51
cc     41
m      38
x      38
d      30
e      25
j      10
r       3
Name: A6, dtype: int64

In [22]:
credit_data['A6'].replace([None], 'c', inplace=True)
credit_data['A6'].value_counts()

c     146
q      78
w      64
i      59
aa     54
ff     53
k      51
cc     41
m      38
x      38
d      30
e      25
j      10
r       3
Name: A6, dtype: int64

In [23]:
credit_data['A7'].value_counts()

v     399
h     138
bb     59
ff     57
j       8
z       8
dd      6
n       4
o       2
Name: A7, dtype: int64

In [24]:
credit_data['A7'].replace([None], 'v', inplace=True)
credit_data['A7'].value_counts()

v     408
h     138
bb     59
ff     57
j       8
z       8
dd      6
n       4
o       2
Name: A7, dtype: int64

Now we will replacing the `Null` value with the `median` for numerical features.

In [25]:
credit_data['A2'].median()

28.46

In [26]:
credit_data['A2'].replace([None], credit_data['A2'].median(), inplace=True)

In [27]:
credit_data['A14'].median()

160.0

In [28]:
credit_data['A14'].replace([None], credit_data['A14'].median(), inplace=True)

Now we will check to make sure all the `Null` values have been replaced.

In [29]:
credit_data.isna().sum()

A1     0
A2     0
A3     0
A4     0
A5     0
A6     0
A7     0
A8     0
A9     0
A10    0
A11    0
A12    0
A13    0
A14    0
A15    0
A16    0
dtype: int64

### Discretizing a feature

This part of the assignment requires us to divide the feature into three bins `Low`, `Medium` and `High`.

Before we do any major changes to the data set it’s good to create a copy to make these changes on. This way if there is any mistakes that can’t easily be fixed you can always replace values and/or features from the cleanest data set


In [30]:
credit_data_copy = credit_data.copy()

credit_data_copy['A2'] = pd.qcut(credit_data_copy['A2'], q=3, labels=['Low', 'Medium', 'High'])

In [31]:
credit_data_copy.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,Medium,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,High,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,Medium,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,Medium,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,Low,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


### Encoding features.

`Scikit-Learn` requires all the values of a dataset to be numerical to run any machine learning algorithms on it. Therefore we need to do encoding on the the catagorical features. 

For the `A2` feature we can use what is known as `integer-encoding`. This will turn catagorical values that have a distinct ranking (`Low`, `Medium` and `High`) into numbers reflect that ranking system. With the other features We don't know what there relationship to each other are so they are considered discriptive features so we will use one-hot-encoding.

### integer-encoding

first we'll start with the `A2` feature.

In [32]:
level_mapping = {'Low': 0, 'Medium': 1, 'High': 2}

credit_data_integer = credit_data_copy.copy()

credit_data_integer['A2'] = credit_data_integer['A2'].replace(level_mapping)

credit_data_integer.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,1,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,2,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,1,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,1,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,0,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


### one-hot-encoding

Next we will move on to one-hot-encoding. First we'll start with the features that only have binary values, This will include the target feature `A16` since this will also need to become an number eventually. 

In [33]:
credit_data_encoded = credit_data_integer.copy()

credit_data_encoded_cols = credit_data_encoded.columns[credit_data_copy.dtypes==object].tolist()

for col in credit_data_encoded_cols:
    n = len(credit_data_encoded[col].unique())
    if (n == 2):
        credit_data_encoded[col] = pd.get_dummies(credit_data_encoded[col], drop_first=True)
        
credit_data_encoded.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,1,0.000,u,g,w,v,1.25,1,1,1,0,g,202.0,0,0
1,0,2,4.460,u,g,q,h,3.04,1,1,6,0,g,43.0,560,0
2,0,1,0.500,u,g,q,h,1.50,1,0,0,0,g,280.0,824,0
3,1,1,1.540,u,g,w,v,3.75,1,1,5,1,g,100.0,3,0
4,1,0,5.625,u,g,w,v,1.71,1,0,0,0,s,120.0,0,0


Now we will encode the catagorical features with mulitple values.

In [34]:
credit_data_encoded = pd.get_dummies(credit_data_encoded)
credit_data_encoded.head()

,A1,A2,A3,A8,A9,A10,A11,A12,A14,A15,...,A7_ff,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s
0,1,1,0.000,1.25,1,1,1,0,202.0,0,...,0,0,0,0,0,1,0,1,0,0
1,0,2,4.460,3.04,1,1,6,0,43.0,560,...,0,1,0,0,0,0,0,1,0,0
2,0,1,0.500,1.50,1,0,0,0,280.0,824,...,0,1,0,0,0,0,0,1,0,0
3,1,1,1.540,3.75,1,1,5,1,100.0,3,...,0,0,0,0,0,1,0,1,0,0
4,1,0,5.625,1.71,1,0,0,0,120.0,0,...,0,0,0,0,0,1,0,0,0,1


As you can now see there are new features added along the dataframe. This is what we want to do but the machine learning algoriths require our target column to be at the end of the datafram ad also to be labelled as `target`.

In [35]:
credit_data_encoded.rename(columns={"A16": "target"}, inplace=True)
cols = list(credit_data_encoded.columns.values)
cols.pop(cols.index('target'))
credit_data_target = credit_data_encoded[cols+['target']]
credit_data_target.head()


,A1,A2,A3,A8,A9,A10,A11,A12,A14,A15,...,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s,target
0,1,1,0.000,1.25,1,1,1,0,202.0,0,...,0,0,0,0,1,0,1,0,0,0
1,0,2,4.460,3.04,1,1,6,0,43.0,560,...,1,0,0,0,0,0,1,0,0,0
2,0,1,0.500,1.50,1,0,0,0,280.0,824,...,1,0,0,0,0,0,1,0,0,0
3,1,1,1.540,3.75,1,1,5,1,100.0,3,...,0,0,0,0,1,0,1,0,0,0
4,1,0,5.625,1.71,1,0,0,0,120.0,0,...,0,0,0,0,1,0,0,0,1,0


### Normalizing the data

It is important to consider normalizing the data as some features could be measured in percentages, years, etc... For this assignment we are normalizing the data using standard scaling. From the `crx.name` file we know that `A2`, `A3`, `A8`, `A11`, `A14` and `A15` are our continous numerical values. So these are the features we will do our normalization on. 

In [36]:
credit_data_norm = preprocessing.StandardScaler().fit_transform(credit_data_target[['A3','A8','A11','A14','A15']])
credit_data_round= credit_data_norm.round(3)
credit_data_round

credit_data_target[["A3","A8","A11","A14","A15"]] = credit_data_round.tolist()
credit_data_target.head()

,A1,A2,A3,A8,A9,A10,A11,A12,A14,A15,...,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s,target
0,1,1,-0.957,-0.291,1,1,-0.288,0,0.107,-0.195,...,0,0,0,0,1,0,1,0,0,0
1,0,2,-0.060,0.244,1,1,0.741,0,-0.817,-0.088,...,1,0,0,0,0,0,1,0,0,0
2,0,1,-0.856,-0.216,1,0,-0.494,0,0.560,-0.037,...,1,0,0,0,0,0,1,0,0,0
3,1,1,-0.647,0.457,1,1,0.535,1,-0.486,-0.195,...,0,0,0,0,1,0,1,0,0,0
4,1,0,0.174,-0.154,1,0,-0.494,0,-0.369,-0.195,...,0,0,0,0,1,0,0,0,1,0


The data set should now be cleaned and ready for processing. As part of the assignmetn we will be naming the cleaned dataframe as `df_clean` then running the required code block three times and then saving a the data fram as a .csv file.

In [37]:
df_clean = credit_data_target

df_clean.shape
df_clean.describe(include='all').round(3) 
df_clean.head(5)

,A1,A2,A3,A8,A9,A10,A11,A12,A14,A15,...,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s,target
0,1,1,-0.957,-0.291,1,1,-0.288,0,0.107,-0.195,...,0,0,0,0,1,0,1,0,0,0
1,0,2,-0.060,0.244,1,1,0.741,0,-0.817,-0.088,...,1,0,0,0,0,0,1,0,0,0
2,0,1,-0.856,-0.216,1,0,-0.494,0,0.560,-0.037,...,1,0,0,0,0,0,1,0,0,0
3,1,1,-0.647,0.457,1,1,0.535,1,-0.486,-0.195,...,0,0,0,0,1,0,1,0,0,0
4,1,0,0.174,-0.154,1,0,-0.494,0,-0.369,-0.195,...,0,0,0,0,1,0,0,0,1,0


In [38]:
df_clean.shape
df_clean.describe(include='all').round(3) 
df_clean.head(5)

,A1,A2,A3,A8,A9,A10,A11,A12,A14,A15,...,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s,target
0,1,1,-0.957,-0.291,1,1,-0.288,0,0.107,-0.195,...,0,0,0,0,1,0,1,0,0,0
1,0,2,-0.060,0.244,1,1,0.741,0,-0.817,-0.088,...,1,0,0,0,0,0,1,0,0,0
2,0,1,-0.856,-0.216,1,0,-0.494,0,0.560,-0.037,...,1,0,0,0,0,0,1,0,0,0
3,1,1,-0.647,0.457,1,1,0.535,1,-0.486,-0.195,...,0,0,0,0,1,0,1,0,0,0
4,1,0,0.174,-0.154,1,0,-0.494,0,-0.369,-0.195,...,0,0,0,0,1,0,0,0,1,0


In [39]:
df_clean.shape
df_clean.describe(include='all').round(3) 
df_clean.head(5)

,A1,A2,A3,A8,A9,A10,A11,A12,A14,A15,...,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A13_g,A13_p,A13_s,target
0,1,1,-0.957,-0.291,1,1,-0.288,0,0.107,-0.195,...,0,0,0,0,1,0,1,0,0,0
1,0,2,-0.060,0.244,1,1,0.741,0,-0.817,-0.088,...,1,0,0,0,0,0,1,0,0,0
2,0,1,-0.856,-0.216,1,0,-0.494,0,0.560,-0.037,...,1,0,0,0,0,0,1,0,0,0
3,1,1,-0.647,0.457,1,1,0.535,1,-0.486,-0.195,...,0,0,0,0,1,0,1,0,0,0
4,1,0,0.174,-0.154,1,0,-0.494,0,-0.369,-0.195,...,0,0,0,0,1,0,0,0,1,0


In [40]:
df_clean.to_csv('df_clean.csv',index=False)

***
# Question 2

Before attempting the questions We will be importing the supplied Corruption Perception Index(CPI) .csv file. Most of these questions attempt to find the CPI of Russia. So I will remove Russia from the original dataframe. Also since we are looking for the CPI for Russia we need to make sure that the placeholder `?` is a `Null` value.

In [41]:
cpi_index = pd.read_csv('Assignment1_Q2.csv', na_values=["?"])

In [42]:
cpi_index.tail(1)

,COUNTRY_ID,LIFE_EXP,TOP10_INCOME,INFANT_MORT,MIL_SPEND,SCHOOL_YEARS,CPI
16,Russia,67.62,31.68,10.0,3.87,12.9,NaN


In [43]:
cpi_index_remove = cpi_index[cpi_index.COUNTRY_ID !='Russia']

In [44]:
cpi_index_remove.tail(1)

,COUNTRY_ID,LIFE_EXP,TOP10_INCOME,INFANT_MORT,MIL_SPEND,SCHOOL_YEARS,CPI
15,New Zealand,80.67,27.81,4.9,1.13,12.3,9.4627


In [45]:
cpi_index_rus = cpi_index[cpi_index.COUNTRY_ID =='Russia']

In [46]:
cpi_index_rus.head()

,COUNTRY_ID,LIFE_EXP,TOP10_INCOME,INFANT_MORT,MIL_SPEND,SCHOOL_YEARS,CPI
16,Russia,67.62,31.68,10.0,3.87,12.9,NaN


### Part a 

`What value would a 3-nearest neighbor prediction model using
Manhattan distance return for the CPI of Russia?`

The formula for Manhattan distance is shown below:

$$ Manhattan(a,b) = \sum_{i=1}^m abs(a[i] - b[i])$$

In order get the Manhattan distance for all rows in the dataframe we will balance all the Russian values into one array and then the values from the main dataframe into another.


In [47]:
rus_array = np.array(cpi_index_rus.loc[:,'LIFE_EXP':'SCHOOL_YEARS'])

In [48]:
test_array = np.array(cpi_index_remove.loc[:,'LIFE_EXP':'SCHOOL_YEARS'])

Here is a quick breakdown of each part of the formaula the following bits of code are working on:

$$ abs(a[i] - b[i])$$


In [49]:
sub_array = abs(rus_array - test_array)

$$ \sum_{i=1}^m $$

In [50]:
sum_array = np.sum(sub_array, axis=1)

This should now be all that is needed to have the Manhattan distance for each country in relation to Russia.

Now we will create a new dataframe to with the `COUNTRY_ID`, `CPI` and the new `MANHATTAN` distance respective to each country.

In [51]:
cpi_manhattan = cpi_index_remove[['COUNTRY_ID','CPI']].copy()
cpi_manhattan["MANHATTAN"] = sum_array.tolist()
cpi_manhattan.head()

,COUNTRY_ID,CPI,MANHATTAN
0,Afghanistan,1.5171,93.85
1,Haiti,1.7999,114.99
2,Nigeria,2.4493,107.07
3,Egypt,2.8622,27.17
4,Argentina,2.9961,17.98


we will now sort by top 3 countries by Manhattan distance.

In [52]:
cpi_manhattan.sort_values(by=['MANHATTAN'])

,COUNTRY_ID,CPI,MANHATTAN
4,Argentina,2.9961,17.98
8,U.S.A,7.1357,18.07
5,China,3.6356,21.07
10,U.K.,7.7751,22.64
15,New Zealand,9.4627,25.36
7,Israel,5.8069,26.26
3,Egypt,2.8622,27.17
9,Ireland,7.5360,28.15
12,Canada,8.6725,29.11
6,Brazil,3.7741,29.39


In [53]:
cpi_closest_3 = cpi_manhattan.sort_values(by=['MANHATTAN'])[:3]

As we can see from above the 3 closest countries to Russia by Manhattan distance are `Argentina`, `U.S.A` and `China`.

To predict Russia's CPI we need to sum the top 3 countries and divide by 3.

In [54]:
rus_cpi = cpi_closest_3['CPI'].sum(axis=0)/3

### The answer to part a:

In [55]:
rus_cpi

4.589133333333334

### Part b

`What value would a weighted k-NN prediction model return for the CPI of Russia? Use k = 16 (i.e., the full dataset) and a weighting scheme of the reciprocal of the squared Manhattan distance between the neighbor and the query.`

The weighted for a weighted k nearest neighbor model uses the  following formula:

$$M_k (q) = argmax l \epsilon levels(t) \sum_{i=1}^k \frac{1}{dist(q,d)^2} x \delta (t_i,l)$$

So to get the weighted part we use:

$$ \frac{1}{dist(q,d)^2} $$

In [56]:
Manhattan_array = np.array(cpi_manhattan['MANHATTAN'])

In [57]:
weighted_array = 1/(Manhattan_array)**2

In [58]:
weighted_array

array([1.13535440e-04, 7.56275188e-05, 8.72297032e-05, 1.35463011e-03,
       3.09328991e-03, 2.25253179e-03, 1.15771278e-03, 1.45014208e-03,
       3.06255358e-03, 1.26195306e-03, 1.95095456e-03, 9.65068292e-04,
       1.18009125e-03, 1.11408001e-03, 8.85243966e-04, 1.55489656e-03])

Now we need t times the weighted results by the taget value (CPI)

In [59]:
cpi_array = np.array(cpi_manhattan['CPI'])

In [60]:
weight_by_cpi = weighted_array * cpi_array

In [61]:
weight_by_cpi

array([0.00017224, 0.00013612, 0.00021365, 0.00387722, 0.00926781,
       0.0081893 , 0.00436932, 0.00842083, 0.02185346, 0.00951008,
       0.01516887, 0.00776504, 0.01023434, 0.00985315, 0.00823144,
       0.01471352])

In [62]:
cpi_manhattan["WEIGHT"] = weighted_array.tolist()
cpi_manhattan["WEIGHTxCPI"] = weight_by_cpi.tolist()
cpi_manhattan

,COUNTRY_ID,CPI,MANHATTAN,WEIGHT,WEIGHTxCPI
0,Afghanistan,1.5171,93.85,0.000114,0.000172
1,Haiti,1.7999,114.99,0.000076,0.000136
2,Nigeria,2.4493,107.07,0.000087,0.000214
3,Egypt,2.8622,27.17,0.001355,0.003877
4,Argentina,2.9961,17.98,0.003093,0.009268
5,China,3.6356,21.07,0.002253,0.008189
6,Brazil,3.7741,29.39,0.001158,0.004369
7,Israel,5.8069,26.26,0.001450,0.008421
8,U.S.A,7.1357,18.07,0.003063,0.021853
9,Ireland,7.5360,28.15,0.001262,0.009510


Now to get our predicted CPI value for Russia, we sum up both the `WEIGHT` feature and the `WEIGHTxCPI` feature. 

In [63]:
np.sum(weight_by_cpi) 

0.13197639799315705

In [64]:
np.sum(weighted_array)

0.021559540605051625

We then divide the sum of the `WEIGHTxCPI` feature by the sum of the `WEIGHT` feature to get our CPI prediction for Russia.

In [65]:
weighted_rus_cpi = np.sum(weight_by_cpi)  / np.sum(weighted_array)


### The answer to part b:

In [66]:
weighted_rus_cpi

6.121484701869464

### Part c

`The descriptive features in this dataset are of different types. For example, some are percentages, others are measured in years, and others are measured in counts per 1,000. We should always consider normalizing our data, but it is particularly important to do this when the descriptive features are measured in different units. What value would a 3-nearest neighbor prediction model using Manhattan distance return for the CPI of Russia when the descriptive features have been normalized using range normalization?`

In order to normalize the data we would use the following formula:

$$ \acute{a}_i = \frac {a_i - min(a)}{max(a)-min(a)} x (high - low) + low $$

We will go through the dataframe without Russia first and normalize it.

In [67]:
nom_cpi_manhattan = cpi_index_remove[['COUNTRY_ID','CPI']].copy()

life_exp_array = np.array(cpi_index_remove['LIFE_EXP'])
nom_life_exp_array = (life_exp_array - np.min(life_exp_array))/(np.max(life_exp_array) - np.min(life_exp_array))*(1-0)+0
nom_cpi_manhattan["LIFE_EXP"] = nom_life_exp_array.tolist()

top10_array = np.array(cpi_index_remove['TOP10_INCOME'])
nom_top10_array = (top10_array - np.min(top10_array))/(np.max(top10_array) - np.min(top10_array))*(1-0)+0
nom_cpi_manhattan["TOP10_INCOME"] = nom_top10_array.tolist()

infant_array = np.array(cpi_index_remove['INFANT_MORT'])
nom_infant_array = (infant_array - np.min(infant_array))/(np.max(infant_array) - np.min(infant_array))*(1-0)+0
nom_cpi_manhattan["INFANT_MORT"] = nom_infant_array.tolist()

mil_array = np.array(cpi_index_remove['MIL_SPEND'])
nom_mil_array = (mil_array - np.min(mil_array))/(np.max(mil_array) - np.min(mil_array))*(1-0)+0
nom_cpi_manhattan["MIL_SPEND"] = nom_mil_array.tolist()

school_array = np.array(cpi_index_remove['SCHOOL_YEARS'])
nom_school_array = (school_array - np.min(school_array))/(np.max(school_array) - np.min(school_array))*(1-0)+0
nom_cpi_manhattan["SCHOOL_YEARS"] = nom_school_array.tolist()

nom_cpi_manhattan.head(5)


,COUNTRY_ID,CPI,LIFE_EXP,TOP10_INCOME,INFANT_MORT,MIL_SPEND,SCHOOL_YEARS
0,Afghanistan,1.5171,0.393907,0.044531,0.896509,0.651198,0.000000
1,Haiti,1.7999,0.000000,1.000000,0.881546,0.000000,0.217391
2,Nigeria,2.4493,0.169857,0.631250,1.000000,0.146707,0.268116
3,Egypt,2.8622,0.686978,0.176172,0.214464,0.264970,0.355072
4,Argentina,2.9961,0.829604,0.399609,0.135910,0.100299,0.702899


Now we will do the same for the seperate Russian dataframe. 

In [68]:
rus_nom_cpi_manhattan = cpi_index_rus[['COUNTRY_ID','CPI']].copy()

rus_nom_life_exp_array = (cpi_index_rus['LIFE_EXP'] - np.min(life_exp_array))/(np.max(life_exp_array) - np.min(life_exp_array))*(1-0)+0
rus_nom_cpi_manhattan["LIFE_EXP"] = rus_nom_life_exp_array.tolist()

rus_nom_top10_array = (cpi_index_rus['TOP10_INCOME'] - np.min(top10_array))/(np.max(top10_array) - np.min(top10_array))*(1-0)+0
rus_nom_cpi_manhattan["TOP10_INCOME"] = rus_nom_top10_array.tolist()

rus_nom_infant_array = (cpi_index_rus['INFANT_MORT'] - np.min(infant_array))/(np.max(infant_array) - np.min(infant_array))*(1-0)+0
rus_nom_cpi_manhattan["INFANT_MORT"] = rus_nom_infant_array.tolist()

rus_nom_mil_array = (cpi_index_rus['MIL_SPEND'] - np.min(mil_array))/(np.max(mil_array) - np.min(mil_array))*(1-0)+0
rus_nom_cpi_manhattan["MIL_SPEND"] = rus_nom_mil_array.tolist()

rus_nom_school_array = (cpi_index_rus['SCHOOL_YEARS'] - np.min(school_array))/(np.max(school_array) - np.min(school_array))*(1-0)+0
rus_nom_cpi_manhattan["SCHOOL_YEARS"] = rus_nom_school_array.tolist()

rus_nom_cpi_manhattan.head()

,COUNTRY_ID,CPI,LIFE_EXP,TOP10_INCOME,INFANT_MORT,MIL_SPEND,SCHOOL_YEARS
16,Russia,NaN,0.609868,0.375391,0.094763,0.565868,0.905797


Here we will follow the same steps as we did in part a but now we are using normalised data.

In [69]:
nom_rus_array = np.array(rus_nom_cpi_manhattan.loc[:,'LIFE_EXP':'SCHOOL_YEARS'])

In [70]:
nom_array = np.array(nom_cpi_manhattan.loc[:,'LIFE_EXP':'SCHOOL_YEARS'])

In [71]:
nom_sub_array = abs(nom_rus_array - nom_array)

In [72]:
nom_sum_array = np.sum(nom_sub_array, axis=1)

In [73]:
nom_sum_array

array([2.33969263, 3.27553437, 2.6579499 , 1.24765207, 0.95356907,
       1.06645103, 1.42216374, 1.02425031, 0.5964457 , 1.18367265,
       0.72950718, 1.24514237, 1.15417553, 1.11011129, 1.23466233,
       1.02026766])

In [74]:
nom_cpi_manhattan = cpi_index_remove[['COUNTRY_ID','CPI']].copy()
nom_cpi_manhattan["MANHATTAN"] = nom_sum_array.tolist()
nom_cpi_manhattan.head()


,COUNTRY_ID,CPI,MANHATTAN
0,Afghanistan,1.5171,2.339693
1,Haiti,1.7999,3.275534
2,Nigeria,2.4493,2.657950
3,Egypt,2.8622,1.247652
4,Argentina,2.9961,0.953569


Now we have our Manhattan distances based on the new normalised data. We will now sort the list to see which are the top three countries closest to Russia are now.

In [75]:
nom_cpi_closest_3 = nom_cpi_manhattan.sort_values(by=['MANHATTAN'])[:3]

In [76]:
nom_cpi_closest_3.head()

,COUNTRY_ID,CPI,MANHATTAN
8,U.S.A,7.1357,0.596446
10,U.K.,7.7751,0.729507
4,Argentina,2.9961,0.953569


This time the top 3 closest countries to Russia are ` U.S.A`, `U.K.` and `Argnetina`. In part a the top 3 countries `Argentina`, `U.S.A` and `China`.

So we will now use the new top three countries to predict Russia CPI using the same method from part a: 

In [77]:
nom_rus_cpi = nom_cpi_closest_3['CPI'].sum(axis=0)/3

### The answer to part c:

In [78]:
nom_rus_cpi

5.968966666666667

### Part d

`What value would a weighted k-NN prediction model—with k = 16
(i.e., the full dataset) and using a weighting scheme of the reciprocal of the squared Euclidean distance between the neighbor and the query—return for the CPI of Russia when it is applied to the range-normalized data?`

Here we will be doing the same as part b but we will use the Normalized dataframe instead.

In [79]:
nom_Manhattan_array = np.array(nom_cpi_manhattan['MANHATTAN'])
nom_weighted_array = 1/(nom_Manhattan_array)**2
nom_cpi_array = np.array(nom_cpi_manhattan['CPI'])
nom_weight_by_cpi = nom_weighted_array * nom_cpi_array

In [80]:
nom_cpi_manhattan["WEIGHT"] = nom_weighted_array.tolist()
nom_cpi_manhattan["WEIGHTxCPI"] = nom_weight_by_cpi.tolist()
nom_cpi_manhattan

,COUNTRY_ID,CPI,MANHATTAN,WEIGHT,WEIGHTxCPI
0,Afghanistan,1.5171,2.339693,0.182676,0.277138
1,Haiti,1.7999,3.275534,0.093204,0.167758
2,Nigeria,2.4493,2.657950,0.141549,0.346696
3,Egypt,2.8622,1.247652,0.642411,1.838709
4,Argentina,2.9961,0.953569,1.099754,3.294974
5,China,3.6356,1.066451,0.879262,3.196644
6,Brazil,3.7741,1.422164,0.494425,1.866011
7,Israel,5.8069,1.024250,0.953208,5.535185
8,U.S.A,7.1357,0.596446,2.810983,20.058329
9,Ireland,7.5360,1.183673,0.713735,5.378704


In [81]:
np.sum(nom_weight_by_cpi) 
np.sum(nom_weighted_array)
nom_weighted_rus_cpi = np.sum(nom_weight_by_cpi)  / np.sum(nom_weighted_array)


### The answer to part d

In [82]:
nom_weighted_rus_cpi

6.609054843134553

### part e

`The actual 2011 CPI for Russia was 2.4488. Which of the predictions made was the most accurate? Why do you think this was?`

Prediction that was the closest was part a. But according to the answers in the textbook when they used Euclidean distance it was part d. They explained that Normalization should help a data set like this since there is a wide range of data ranges that are very different from each other, therefore data normalization should be very important and help provide more accurate results. in part c we can see that it did provided a better prediction than part b but still not close to what the proper prediction should be. 

Also, weighting should have an impact on this dataset due to it being a small dataset.

Overall, I don't know why Manhattan distance is providing a worse prediction than using Euclidian distance.
